### Appendix C - Model Predictive Control

The following code heavily realies on the source:
https://www.nicholasmoehle.com/talks/codegen.pdf

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import codegen

In [2]:
np.random.seed(0)
n = 5
A = np.zeros([n,n])
for j in range(5):
    A[0] = [np.random.random()*(1+int(x==0)) for x in range(5)]
    A[0] = A[0]/np.sum(A[0])

for i in range(1,5):
    A[i] =[np.random.random()*3]+[np.random.random()*(3-int((abs(i-j)+1)/2)) for j in range(1,5)]
    A[i] = A[i]/np.sum(A[i])

In [3]:
np.random.seed(0)
O = np.array([42,17,13,20,8])*5
d = np.round(np.random.normal(0, 15, 5))
x0 = np.round((O+d)/np.sum(O+d)*500)
B = np.zeros([n**2,n])
for i in range(n**2):
    for j in range(n):
        a = int(i/n)
        b = (i)%n
        if j == a:
            if b != a:
                B[i][j] = -1
        if j == b:
            if b != a:
                B[i][j] = 1

In [7]:
x0 = np.array([50,10,20,10,10])
u_max = 25
u_min = 0
O = np.array([42,17,13,20,8])*5
T = 5

In [8]:
def MPC(x_k, T=5):
    x = cp.Variable((n, T+1))
    u = cp.Variable((n**2, T))
    obj = 0
    constr = [x[:,0] == x_k, x[:,-1] == 0]
    for t in range(T):
        constr += [x[:,t+1] == A*x[:,t] + B.transpose()*u[:,t], cp.norm(u[:,t], float('inf')) <= u_max]
        constr += [u[:,t]>=u_min]
        constr += [np.ones(n**2)@u[:,t] <= 100]
        obj += cp.sum_squares(x[:,t]-O) + np.ones(n**2)*(u[:,t])

    prob = cp.Problem(cp.Minimize(obj), constr)

    solution = prob.solve()
    sol = np.round(u.value)
    return sol, solution

In [9]:
sol, solution = MPC(x0)

In [10]:
def print_us(sol,T=5):
    for i in range(T):
        print('\n\nT = {} , u_{}|k'.format(i,i))
        display(pd.DataFrame(sol.transpose()[i].reshape(n,n)))

In [11]:
print_us(sol)



T = 0 , u_0|k


,0,1,2,3,4
0,-0.0,-0.0,-0.0,-0.0,-0.0
1,0.0,-0.0,0.0,0.0,-0.0
2,25.0,15.0,-0.0,12.0,5.0
3,2.0,-0.0,-0.0,-0.0,-0.0
4,2.0,-0.0,0.0,0.0,-0.0




T = 1 , u_1|k


,0,1,2,3,4
0,-0.0,6.0,-0.0,-0.0,7.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,4.0,25.0,-0.0,12.0,25.0
3,-0.0,3.0,-0.0,-0.0,6.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 2 , u_2|k


,0,1,2,3,4
0,-0.0,11.0,-0.0,-0.0,22.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,2.0,25.0,-0.0,9.0,25.0
3,-0.0,0.0,-0.0,-0.0,6.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 3 , u_3|k


,0,1,2,3,4
0,-0.0,25.0,-0.0,0.0,21.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,-0.0,25.0,-0.0,4.0,25.0
3,-0.0,-0.0,-0.0,-0.0,-0.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 4 , u_4|k


,0,1,2,3,4
0,-0.0,-0.0,-0.0,-0.0,-0.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,2.0,0.0,-0.0,-0.0,-0.0
3,3.0,8.0,-0.0,-0.0,-0.0
4,3.0,7.0,-0.0,-0.0,-0.0


In [12]:
x1_pred = np.dot(A,x0)+np.dot(sol.transpose()[0],B) 
x1_pred

array([ 59.1386304 ,  41.79395806, -37.69992237,  30.16417311,
        24.34707095])

In [13]:
sol1, solution1 = MPC(x1_pred)

In [14]:
print_us(sol1)



T = 0 , u_0|k


,0,1,2,3,4
0,0.0,-0.0,-0.0,-0.0,-0.0
1,0.0,0.0,0.0,0.0,-0.0
2,20.0,25.0,0.0,3.0,9.0
3,0.0,-0.0,0.0,0.0,-0.0
4,0.0,0.0,0.0,0.0,0.0




T = 1 , u_1|k


,0,1,2,3,4
0,-0.0,6.0,-0.0,-0.0,7.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,4.0,25.0,-0.0,12.0,25.0
3,-0.0,3.0,-0.0,-0.0,7.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 2 , u_2|k


,0,1,2,3,4
0,-0.0,11.0,-0.0,-0.0,23.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,2.0,25.0,-0.0,9.0,25.0
3,-0.0,0.0,-0.0,-0.0,6.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 3 , u_3|k


,0,1,2,3,4
0,-0.0,25.0,-0.0,0.0,21.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,-0.0,25.0,-0.0,4.0,25.0
3,-0.0,-0.0,-0.0,-0.0,-0.0
4,-0.0,-0.0,-0.0,-0.0,-0.0




T = 4 , u_4|k


,0,1,2,3,4
0,-0.0,-0.0,-0.0,-0.0,-0.0
1,-0.0,-0.0,-0.0,-0.0,-0.0
2,2.0,-0.0,-0.0,-0.0,-0.0
3,3.0,8.0,-0.0,-0.0,-0.0
4,3.0,7.0,-0.0,-0.0,-0.0
